[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vectara/example-notebooks/blob/main/notebooks/hackathon-2023-code-samples.ipynb)

# RAG hackathon - sample code

In this notebook we share some of the code for the RAG hackathon tutorial.<br>
It is not meant to be run top to bottom as some of the code is "demonstrative".

## Indexing text into Vectara

This code snippet shows how to create a dictionary for the document object, including documentId, title, metadataJson and the section array which includes the text. Then we use `requests` to send a HTTP POST request for the indexing operation


In [ ]:
api_endpoint = "https://api.vectara.io/v1/index"
document = {
  "documentId": doc_id,
  "title": doc_title,
  "metadataJson": json.dumps(metadata_dict),
  "section": [
    {'text': "some initial text", "title": "title of first part"},
    {'text': "some other text", "title": "title of 2nd part"}
  ]
}
request = {
  "customer_id": "<YOUR-CUSTOMER-ID>",
  "corpus_id": "<YOUR-CORPUS-ID>",
  "document": document
}
post_headers = {
  "x-api-key": "<YOUR-API-KEY>",
  "customer_id": "<YOUR-CUSTOMER-ID>"
}
response = requests.post(api_endpoint, data=json.dumps(reqeust), verify=True, headers=post_headers)


## Using JWT token instead of API-key

If you'd like to use a JWT token instead of the api-key, just replace the post-headers as follows:

In [ ]:
post_headers = {
  "Authorization": f"Bearer {jwt_token}",
  "customer_id": "<YOUR-CUSTOMER-ID>"
}
response = requests.post(api_endpoint, data=json.dumps(reqeust), verify=True, headers=post_headers)


## File Upload API call

Vectara also supports a method to directly upload a file via the API, including support for various file types like DOC/DOCX, PPT/PPTX, PDF, HTML, JSON and many others.

In [ ]:
customer_id = "<YOUR-CUSTOMER-ID>"
corpus_id = "<YOUR-CORPUS-ID>"
api_key = "<YOUR-API-KEY>"
url = f"https://api.vectara.io/v1/upload?c={customer_id}&o={corpus_id}"

post_headers = { 
    "x-api-key": api_key,
    "customer-id": customer_id
}
files: Any = {
    "file": (filename, open(filename, 'rb')),
    "doc_metadata": (None, json.dumps(doc_metadata)),
}  
response = requests.post(url, files=files, verify=True, headers=post_headers)

## Query

This code sample in Javascript demonstrates a call to Vectara's query API.
This includes the optional `summary` array which, if exists, defines a generative summary request parameters.

In [ ]:
const customer_id = "<YOUR-CUSTOMER-ID>"
const corpus_id = "<YOUR-CORPUS-ID>"
const api_key = "<YOUR-API-KEY>"

const query_body = {
    query: [
        {
            query: query_str,
            start: 0,
            numResults: 10,
            corpusKey: [{
                customerId: customer_id, 
                corpusId: corpus_id, 
                lexicalInterpolationConfig: {lambda: 0.025}
            }],
            contextConfig: {
                sentencesBefore: 3,
                sentencesAfter: 3,
                startTag: "%START_TAG%",
                endTag: "%END_TAG%"
            },
            summary: [
                {
                    responseLang: "eng",
                    maxSummarizedResults: 7,
                    summarizerPromptName: "vectara-summarizer-ext-v1.3.0"
                }
            ]
        }
    ]
}
const url = "https://api.vectara.io/v1/query";
const post_headers = {
    "headers": {
        "Content-type": "application/json",
        "Accept": "application/json",
        "customer-id": customer_id,
        "x-api-key": api_key 
    }
};
const result = await axios.post(url, body, post_headers)

## Using LlamaIndex chat query engine with Vectara

This example shows how to create a chat flow using Vectara and LlamaIndex.

In [1]:
from llama_index import SimpleDirectoryReader
from llama_index.indices import VectaraIndex

documents = SimpleDirectoryReader("../data/paul_graham").load_data()
index = VectaraIndex.from_documents(documents)

chat_engine = index.as_chat_engine(similarity_top_k=10)
response = chat_engine.chat("what is YC?")
print(response)
response = chat_engine.chat("who started it?")
print(response)

YC stands for Y Combinator. It is a startup accelerator and venture capital firm based in Silicon Valley, California. Y Combinator provides seed funding, mentorship, and resources to early-stage startups. It is known for its highly competitive application process and has helped launch successful companies such as Airbnb, Dropbox, Reddit, and Stripe. Y Combinator also runs a three-month program where selected startups receive funding and guidance to develop their products and business models.
Y Combinator was started by Paul Graham, Robert Morris, Trevor Blackwell, and Jessica Livingston in March 2005. Paul Graham is a computer scientist, entrepreneur, and writer who is known for his work on programming languages and his influential essays on startups and technology. Robert Morris is a computer scientist and entrepreneur who co-authored the Morris Worm, one of the first computer worms to spread across the internet. Trevor Blackwell is a computer scientist and entrepreneur who has worked